# 部署模型
現在我們的模型已經過完整訓練，派上用場的時候到了。在本練習中，我們會讓模型讀取新影像，並偵測手語字母表中的正確字母。現在就開始吧！

## 目標

* 從磁碟載入經過訓練的模型
* 重新格式化影像以適用於以不同格式影像進行訓練的模型
* 讓經過訓練的模型推論未見過的新影像，並且評估其效能

## 載入模型
現在我們使用的是新的 Notebook，所以我們要載入之前訓練過並儲存的模型。我們在先前的練習儲存模型時，建立了一個名稱為 "asl_model" 的資料夾。只要選取相同的資料夾就可以載入模型。

In [ ]:
from tensorflow import keras

model = keras.models.load_model('asl_model')

如果你想要確保一切都沒有變動，可以再次查看模型的摘要。

In [ ]:
model.summary()

## 為模型準備影像

現在要開始使用模型對先前從未見過的全新影像進行預測。這個過程也稱為推論。我們在 asl_images 資料夾中提供了一組影像。請試著用左側導覽面板開啟並探索影像。

你應該會發現，眼前影像的解析度比資料集中的影像高出許多，而且是彩色的。請記得，資料集中的影像是 28x28 像素和灰階。請務必要記得，每次使用模型進行預測時，輸入內容的形狀必須與模型的訓練資料的形狀相符。以這個模型而言，訓練資料集的形狀為：(27455, 28, 28, 1)。意思是有 27,455 張 28x28 像素的影像，每一張皆為單一色頻 (灰階)。

### 顯示影像

當我們使用模型對新影像進行預測，同時顯示影像會很有幫助。我們可以使用 matplotlib 函式庫做到這一點。

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

def show_image(image_path):
    image = mpimg.imread(image_path)
    plt.imshow(image)

In [ ]:
show_image('asl_images/b.png')

### 調整影像

資料集中的影像是 28x28 像素和灰階。我們必須確保傳遞給預測方法的影像也是相同的大小和色彩。有幾種方法可以透過 Python 編輯影像，不過 Keras 的內建公用程式也有同樣的功能。

In [ ]:
from tensorflow.keras.preprocessing import image as image_utils

def load_and_scale_image(image_path):
    image = image_utils.load_img(image_path, color_mode="grayscale", target_size=(28,28))
    return image

In [ ]:
image = load_and_scale_image('asl_images/b.png')
plt.imshow(image, cmap='gray')

### 準備用於預測的影像

現在我們有 28x28 像素的灰階影像，很快就能準備好將影像傳遞到模型中以進行預測。首先，我們需要重新調整影像，以符合用於訓練模型的資料集形狀。在重新調整前，我們需要將影像轉換成更原始的格式。我們要使用名為 image_to_array 的 Keras 公用程式來進行這項操作。

In [ ]:
image = image_utils.img_to_array(image)

現在我們可以重新調整影像，為預測做好準備。

In [ ]:
# This reshape corresponds to 1 image of 28x28 pixels with one color channel
image = image.reshape(1,28,28,1) 

最後，務必要記得正規化資料 (讓所有值都介於 0-1)，就像我們在處理訓練資料集一樣：

In [ ]:
image = image / 255

## 進行預測

現在，我們已經準備好進行預測了！方法是將經過前置處理的影像傳遞至模型的預測方法。

In [ ]:
prediction = model.predict(image)
print(prediction)

### 瞭解預測過程

預測的格式為 24 長度陣列。儘管看起來有點不同，這個格式和 y_train 與 y_test 的「binarized」分類陣列格式相同。陣列中的每個元素都是介於 0 和 1 的機率，表示每個類別的可信度。讓我們用更容易理解的方式說明。首先我們可以找出陣列中哪個元素代表最高的機率。運用 Numpy 函式庫和 [argmax](https://numpy.org/doc/stable/reference/generated/numpy.argmax.html) 函數就能輕易做到這一點。

In [ ]:
import numpy as np
np.argmax(prediction)

預測陣列的每個元素都代表手語字母表中的一個可能字母。別忘了，j 和 z 不是選項，因為表達這兩個字母需要移動手部，但我們只有處理靜態相片。讓我們在預測陣列的索引和對應的字母之間建立對應。

In [ ]:
# Alphabet does not contain j or z because they require movement
alphabet = "abcdefghiklmnopqrstuvwxy"
dictionary = {}
for i in range(24):
    dictionary[i] = alphabet[i]
dictionary

現在我們可以傳遞預測索引來找出對應的字母。

In [ ]:
dictionary[np.argmax(prediction)]

## 練習：整合所有過程

讓我們把所有過程都放入函數，就可以從影像檔案直接進行預測。使用上述的函數和步驟，並在下方的函數中實作。如果需要協助，請按一下下方的三個點以顯示解決方案。

In [ ]:
def predict_letter(file_path):
    # Show image
    FIXME
    # Load and scale image
    image = FIXME
    # Convert to array
    image = FIXME
    # Reshape image
    image = FIXME
    # Normalize image
    image = FIXME
    # Make prediction
    prediction = FIXME
    # Convert prediction to letter
    predicted_letter = FIXME
    # Return prediction
    return predicted_letter   

## 解決方案

按一下以下的「...」來檢視解決方案。

```python
def predict_letter(file_path):
    show_image(file_path)
    image = load_and_scale_image(file_path)
    image = image_utils.img_to_array(image)
    image = image.reshape(1,28,28,1) 
    image = image/255
    prediction = model.predict(image)
    # convert prediction to letter
    predicted_letter = dictionary[np.argmax(prediction)]
    return predicted_letter
```

In [ ]:
predict_letter("asl_images/b.png")

讓我們也測試看看 asl_images 資料集中的字母「a」資料：

In [ ]:
predict_letter("asl_images/a.png")

## 摘要

你在練習中表現得很好！你已經完成了整個流程：從頭開始訓練高度精確的模型，接著使用模型進行全新且有意義的預測。如果你還有時間，建議你使用網路攝影機拍攝圖片，並拖曳到 asl_data 資料夾上傳，然後用來測試模型。如果你是 Mac 使用者，可以使用 Photo Booth。如果你是 Windows 使用者，你可以從「開始」畫面選取相機應用程式。我們希望你可以試試看。這可是學些手語的好機會！例如，用你的姓名拼音字母試試看。

我們可以想像到要如何在應用程式中使用這個模型，例如教人學手語，甚至協助無法說話的人與電腦互動。如果你很熟悉網路開發，甚至可以透過 [TensorFlow](https://www.tensorflow.org/js) 這個函式庫在瀏覽器中使用模型。

## 清除記憶體

在繼續之前，請執行下列儲存格以清除 GPU 記憶體。

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

## 下一步

希望你喜歡這些練習！在接下來的章節中，我們要學習如何在沒有可靠的資料集時善用深度學習。到時候見了！
若要深入瞭解邊緣端推論，請參閱有關這項主題的[精彩白皮書](http://web.eecs.umich.edu/~mosharaf/Readings/FB-ML-Edge.pdf)。

現在你已經熟悉要如何建立自己的模型，也稍微瞭解這些模型的運作方式，接下來我們會把重點放在非常強大的技術，也就是使用預先訓練的模型來加速作業。請繼續前往下一節：[*預先訓練的模型*](./05a_doggy_door.ipynb)。